# Take Home Exam

## Instructions:
* Make sure that you run all code cells that are included in this notebook.
* I will deduct points in **half-point steps** in the following cases:
    * Your code does not work.
    * You are not answering the question (Read carefully).
    * Your code is slow or inefficient.
    * You do not explain why you are doing what you are doing.
    * Clean **unnecessary** elements before submission.
    * **Check the rubric on Camino for this assignment!**
    
**The deadline is Saturday, February 10, 2018, 11:59PM**.

**I am not available on Saturday and have only limited access to my email. If you have questions or issues, make sure you adress them on Friday. **

## Overview

In [2]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.4f}'.format

In [3]:
%matplotlib inline

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
cd = pd.read_csv('complications.csv', dtype=str).rename(columns=lambda x: x.strip())

This dataset contains complications in U.S. hospitals. Each observation represents the number of complications and the 30-day death rate for a specific complication in a specific hospital.

The columns are as follows:

In [6]:
cd.columns

Index(['Provider ID', 'Hospital Name', 'Address', 'City', 'ZIP Code',
       'County Name', 'Phone Number', 'Year', 'Measure ID', 'Measure Name',
       'Denominator', 'Score', 'Lower Estimate', 'Higher Estimate'],
      dtype='object')

| Column Name | Description|
|-------------|------------|
|Provider ID | Unique ID of the hospital |
|Hospital Name| Name of the hospital|
|Address| Address of the hospital|
|City| Name of the city the hospital is in |
|ZIP Code | Postal code of the hospital|
|County Name | Name of the county the hospital is in|
|Phone Number | Phone number of the hospital |
|Year | The year in which the complications were recorded|
|Measure ID | Code for the complication |
|Measure Name | Name of the complication|
|Denominator| Number of underlying medical procedures|
|Score | Estimate for number of serious complications |
|Lower Estimate | Lower boundary of score|
|Higher Estimate | Upper boundary of score|

A quick overview of the dataset:

In [7]:
cd.describe()

,Provider ID,Hospital Name,Address,City,ZIP Code,County Name,Phone Number,Year,Measure ID,Measure Name,Denominator,Score,Lower Estimate,Higher Estimate
count,272788,272788,272788,272788,272788,272626,272788,272788,272788,272788,272788,272788,272788,272788
unique,4746,5119,4915,2874,4368,1526,5028,6,26,26,23771,6373,8690,11699
top,140116,MEMORIAL HOSPITAL,100 HOSPITAL DRIVE,CHICAGO,76104,LOS ANGELES,6104022273,2013,MORT_30_PN,Pneumonia (PN) 30-Day Mortality Rate,26.0,0.06,0.0,0.06
freq,89,710,378,1957,353,5421,178,58668,24942,24942,1435,2947,23907,2897


A sample of the dataset:

In [8]:
cd.sample(5, random_state=1234).head()

,Provider ID,Hospital Name,Address,City,ZIP Code,County Name,Phone Number,Year,Measure ID,Measure Name,Denominator,Score,Lower Estimate,Higher Estimate
22678,050222,SHARP CHULA VISTA MEDICAL CENTER,751 MEDICAL CENTER COURT,CHULA VISTA,91911,SAN DIEGO,6195025800,2016,MORT_30_PN,Pneumonia (PN) 30-Day Mortality Rate,474.0,14.3,12.1,16.8
22088,050191,ST MARY MEDICAL CENTER,1050 LINDEN AVE,LONG BEACH,90813,LOS ANGELES,5624919000,2012,PSI_14_POSTOP_DEHIS,A wound that splits open after surgery on the...,102.0,1.78,0.0,3.64
111295,201309,MAYO REGIONAL HOSPITAL,897 WEST MAIN STREET,DOVER FOXCROFT,04426,PISCATAQUIS,2075644251,2014,MORT_30_STK,Death rate for stroke patients,43.0,14.1,10.2,18.9
44865,100012,LEE MEMORIAL HOSPITAL,2776 CLEVELAND AVE,FORT MYERS,33901,LEE,2393321111,2012,MORT_30_PN,Pneumonia (PN) 30-Day Mortality Rate,971.0,10.6,8.9,12.5
208974,390194,BLUE MOUNTAIN HOSPITAL-GNADEN HUETTEN CAMPUS,211 NORTH 12TH STREET,LEHIGHTON,18235,CARBON,6073771300,2016,MORT_30_STK,Death rate for stroke patients,72.0,15.7,11.7,20.8


# Transform relevant columns

All columns in the dataset are of type `str` (to preserve the ZIP code information). For **five** columns, str is the wrong type. Change the columns and explain why you changed which column.

In [10]:
# copy of original dataframe
cd1= cd.copy()

In [11]:
cd1.Year = pd.to_datetime(cd1.Year)
cd1.Denominator = pd.to_numeric(cd1.Denominator)
cd1.Score = pd.to_numeric(cd1.Score)
cd1['Lower Estimate'] = pd.to_numeric(cd1['Lower Estimate'])
cd1['Higher Estimate'] = pd.to_numeric(cd1['Lower Estimate'])

- Year column should be change to 'datetime' data type, since we could with time calculations easier. For example, we may want to calculate the total number of years that has a score above 3.5<br>
- Denominator, Score, Lower Estimate, and Higher Estimate columns should all be changed to numeric type columns, since we might want to do calculations with those columns.

# Create a ratio between the score and the denominator

Create a new column in the dataset that contains the ratio value.

In [248]:
# Creates an empty column
cd1['ratio'] = cd1.Score/cd1.Denominator

# Create an overview of the ratio for deathly complications

0. **Be careful when you identify 'deathly' complications and make sure that you catch all of them.**
1. Create the overview (Make sure that the overview is understandable to everyone.)
2. Choose an appropriate measure to aggregate the data. **Ignore that the data is for multiple years**.
2. Sort with the highest ratio on top.

In [249]:
# Check for null values in all columns
cd1.isna().any()

Provider ID        False
Hospital Name      False
Address            False
City               False
ZIP Code           False
County Name         True
Phone Number       False
Year               False
Measure ID         False
Measure Name       False
Denominator        False
Score              False
Lower Estimate     False
Higher Estimate    False
ratio              False
dtype: bool

In [250]:
cd1['Measure Name'].unique()

array(['Rate of complications for hip/knee replacement patients',
       'Acute Myocardial Infarction (AMI) 30-Day Mortality Rate',
       'Death rate for CABG',
       'Death rate for chronic obstructive pulmonary disease (COPD) patients',
       'Heart failure (HF) 30-Day Mortality Rate',
       'Pneumonia (PN) 30-Day Mortality Rate',
       'Death rate for stroke patients',
       'Postoperative Acute Kidney Injury Requiring Dialysis Rate',
       'Postoperative Respiratory Failure Rate',
       'Serious blood clots after surgery',
       'Blood stream infection after surgery',
       'A wound that splits open  after surgery on the abdomen or pelvis',
       'Accidental cuts and tears from medical treatment',
       'Pressure sores',
       'Deaths among Patients with Serious Treatable Complications after Surgery',
       'Collapsed lung due to medical treatment',
       'Infections from a large venous catheter',
       'Broken hip from a fall after surgery',
       'Perioperative H

- By examining the unique Measure Name column, we will get an idea of the complication details. Several conditions may be considered "deathly" upon glance:<br>
&nbsp;&nbsp;&nbsp; 'Death rate for CABG'
&nbsp;&nbsp;&nbsp; 'Death rate for chronic obstructive pulmonary disease (COPD) patients'<br>
&nbsp;&nbsp;&nbsp; 'Heart failure (HF) 30-Day Mortality Rate'<br>
&nbsp;&nbsp;&nbsp; 'Pneumonia (PN) 30-Day Mortality Rate'<br>
&nbsp;&nbsp;&nbsp; 'Death rate for stroke patients'<br>
&nbsp;&nbsp;&nbsp; 'Deaths among Patients with Serious Treatable Complications after Surgery'<br><br>

- Several key words can be used to categorize these conditions:"Deaths", "Death", "Mortality" <br>
- Note: mask could not be put on all three key words in the same expression; dtype error result: unsupported operand type(s) for &: 'str' and 'int'




In [252]:
# Tokenize each word in this column to work with the strings
cd1['Measure Name'].str.split()

0         [Rate, of, complications, for, hip/knee, repla...
1         [Rate, of, complications, for, hip/knee, repla...
2         [Rate, of, complications, for, hip/knee, repla...
3         [Rate, of, complications, for, hip/knee, repla...
4         [Rate, of, complications, for, hip/knee, repla...
5         [Rate, of, complications, for, hip/knee, repla...
6         [Acute, Myocardial, Infarction, (AMI), 30-Day,...
7         [Acute, Myocardial, Infarction, (AMI), 30-Day,...
8         [Acute, Myocardial, Infarction, (AMI), 30-Day,...
9         [Acute, Myocardial, Infarction, (AMI), 30-Day,...
10        [Acute, Myocardial, Infarction, (AMI), 30-Day,...
11        [Acute, Myocardial, Infarction, (AMI), 30-Day,...
12                                 [Death, rate, for, CABG]
13                                 [Death, rate, for, CABG]
14                                 [Death, rate, for, CABG]
15        [Death, rate, for, chronic, obstructive, pulmo...
16        [Death, rate, for, chronic, ob

In [154]:
# Filter for data that contains words associated with deathly complications
deathly = cd1[cd1['Measure Name'].str.contains('Death') | cd1['Measure Name'].str.contains('Mortality')|cd1['Measure Name'].str.contains('Deaths') ]

In [81]:
d1 = deathly.groupby(['Measure Name'])

In [87]:
d1.ratio.agg(['min','max','mean']).sort_values(by='mean', ascending=False)

,min,max,mean
Measure Name,,,
Deaths among Patients with Serious Treatable Complications after Surgery,0.1132,0.9997,0.5244
Death rate for stroke patients,0.0070,0.7000,0.1848
Acute Myocardial Infarction (AMI) 30-Day Mortality Rate,0.0068,0.6680,0.1599
Heart failure (HF) 30-Day Mortality Rate,0.0033,0.6360,0.1217
Pneumonia (PN) 30-Day Mortality Rate,0.0044,0.7520,0.1159
Death rate for chronic obstructive pulmonary disease (COPD) patients,0.0028,0.3640,0.0833
Death rate for CABG,0.0016,0.1750,0.0415


# Identify the five safest hospitals

Return a **list** with the hospitals that have, on average, the lowest rate of deadly complications. **Be careful when you identify 'deathly' complications.** (Again, ignore that the dataset covers multiple years).

In [91]:
deathly.groupby('Hospital Name')['ratio'].mean().nsmallest(5)

Hospital Name
UNIVERITY OF MD BALTO WASHINGTON  MEDICAL CENTER   0.0206
NEW YORK-PRESBYTERIAN HOSPITAL                     0.0220
NORTON HOSPITALS, INC                              0.0226
SAN JUAN VA MEDICAL CENTER                         0.0240
FLORIDA HOSPITAL                                   0.0256
Name: ratio, dtype: float64

# Augment the datset with information about the state

Read the file `geo-data.csv` into an object `zip_codes`. **Make sure that you read all columns as string.** ([Link](https://github.com/scpike/us-state-county-zip))

In [103]:
zip_codes = pd.read_csv('geo-data.csv', dtype=str)

This file contains information about the relationship of zip codes, counties, and U.S. states. 

In [104]:
zip_codes.columns

Index(['state_fips', 'state', 'state_abbr', 'zipcode', 'county', 'city'], dtype='object')

| Column Name | Description|
|-------------|------------|
|state_fips | FIPS code for the state |
|state| Name of the state|
|state_abbr| Abbreviation of the state|
|zipcode | Postal code|
|county | County name|
|city | City name  |

1. Combine the dataset on complications with the dateset on zip codes.
2. Explain why you chose the column that used to combine the datasets.
3. **Do not worry, not all states will be matched.**

- First, I will clean up the zip_codes dataframe by selecting only the 'zipcode' and 'state'columns, since these are the only columns we want. I then rename the 'zipcode' column to match the 'ZIP code' column from complications frame. In the actual merge, a left merge is used to only preserve the states which found a match in the complications dataframe (here named 'deathly').<br>

- I chose to join only on the zip code column, because after checking the complications dataframe earlier, I found that there were NaN values in the county column. If I had chosen to join on both zip code and county, there would be rows of data that could be cut off and thereby, affect the integrity of the entire set.

In [253]:
df = zip_codes[['state','zipcode']]
df.columns = ['state','ZIP Code']

In [255]:
merged_df = cd1.merge(df, how = 'left', on = "ZIP Code")


# Create an overview of the complications per U.S. state

* Return a sorted list with the overall complication rate per state. 
* Ignore that the data is for multiple years. 
* Focus on the **best** ten states.

In [256]:
stateComp = merged_df.groupby(['state'])
stateComp.Score.mean().nsmallest(10)

state
Utah           10.1762
New mexico     10.6304
Hawaii         10.7439
Louisiana      10.8218
Pennsylvania   10.8478
Texas          10.8674
South dakota   10.9352
California     10.9853
Wyoming        10.9981
Oklahoma       10.9985
Name: Score, dtype: float64

# Create an overview of the change in complications across U.S. states

* Focus on the deadly complications only.
* Calculate the change as follows: **(average rate for last year in dataset - average rate for first year in dataset)/(average rate for first year in dataset)**

- First, I merged the dataframe with only deathly complications('deathly') with zipcode dataframe, joined on zip code.<br>
- Second, I pulled two individual sets of series; one is on the average complication rate of 2011 for each state, and one is the average complication rate of 2016 for each state<br>
- I then combined these series to create a new dataframe, where I finally calculated the change rate between the two.<br>

In [257]:
merged_df2 = deathly.merge(df, how = 'left', on = "ZIP Code")

In [282]:
group11 = merged_df2[(merged_df2.Year =='2011-01-01')].groupby(['state','Year'])['Score'].mean()
group16 = merged_df2[(merged_df2.Year =='2016-01-01')].groupby(['state','Year'])['Score'].mean()

In [283]:
# Combine both series into one dataframe
newmerged = pd.merge(group11.reset_index(), group16.reset_index(),on='state')

In [284]:
# Calculate the change rate
newmerged['rate'] = (newmerged.Score_y - newmerged.Score_x)/newmerged.Score_x

Which state has improved the most?<br><br>
**Wyoming**

In [285]:
newmerged.nlargest(columns='rate', n=1)

,state,Year_x,Score_x,Year_y,Score_y,rate
43,Wyoming,2011-01-01,12.0308,2016-01-01,13.3884,0.1128


Which state has the worst development?<br><br>
**Nevada**

In [286]:
newmerged.nsmallest(columns='rate', n=1)

,state,Year_x,Score_x,Year_y,Score_y,rate
25,Nevada,2011-01-01,12.7653,2016-01-01,12.2444,-0.0408


# Bonus

Which state has the worst development for 'Deaths among Patients with Serious Treatable Complications after Surgery'?<br><br>
- same process as above, but when data is filtered for the specified measure, only 2012 to 2015 data is available. Process is slightly modified to take this into account; worst state is Hawaii

In [287]:
deathly2 = cd1[cd1['Measure Name'].str.contains('Patients') | cd1['Measure Name'].str.contains('Treatable')|cd1['Measure Name'].str.contains('Surgery') ]
merged_dff = deathly2.merge(df, how = 'left', on = "ZIP Code")


In [293]:
newgroup12 = merged_dff[(merged_dff.Year =='2012-01-01')].groupby(['state','Year'])['Score'].mean()
newgroup15 = merged_dff[(merged_dff.Year =='2015-01-01')].groupby(['state','Year'])['Score'].mean()


In [295]:
# Combine both series into one dataframe
newmerged2 = pd.merge(newgroup12.reset_index(), newgroup15.reset_index(),on='state')


In [297]:
# Calculate the change rate
newmerged2['rate'] = (newmerged2.Score_y - newmerged2.Score_x)/newmerged2.Score_x

In [298]:
newmerged2.nsmallest(columns='rate', n=1)

,state,Year_x,Score_x,Year_y,Score_y,rate
10,Hawaii,2012-01-01,62.9300,2015-01-01,52.6667,-0.1631


### SOURCES:<BR>
   https://www.youtube.com/watch?v=h4hOPGo4UVU - join dataframe <br>
   https://stackoverflow.com/questions/27975069/how-to-filter-rows-containing-a-string-pattern-from-a-pandas-dataframe - string pattern <br>
   https://stackoverflow.com/questions/18062135/combining-two-series-into-a-dataframe-in-pandas - combine series <br>
   https://stackoverflow.com/questions/27925055/join-series-on-multiindex-in-pandas - combine series<br>